In [16]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.contrib.layers import flatten

train_data = pd.read_csv("./fashion-mnist_train.csv")
train_labels = train_data.iloc[:, 0]
train_pixels = train_data.iloc[:,1:785]
train_pixels = train_pixels.values

X_train, y_train = train_pixels[0:50000,:], train_labels[0:50000]
X_val, y_val = train_pixels[50000:60000,:], train_labels[50000:60000]

test_data = pd.read_csv("./fashion-mnist_test.csv")
y_test = test_data.iloc[:, 0]
X_test = test_data.iloc[:,1:785]
X_test = X_test.values

height = 28
width = 28
channels = 1
n_inputs = height * width

n_outputs = 10

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

conv1 = tf.layers.conv2d(X_reshaped, filters=15, kernel_size=3,strides=1,activation=tf.nn.tanh)
conv2 = tf.layers.conv2d(conv1, filters=25, kernel_size=3,strides=1,activation=tf.nn.tanh)
pool1 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
drop = tf.layers.dropout(pool1, .3, training=training)
conv3 = tf.layers.conv2d(drop, filters=15, kernel_size=3,strides=1,activation=tf.nn.tanh)
pool2 = tf.nn.avg_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding="SAME")
conv4 = tf.layers.conv2d(pool2, filters=64, kernel_size=3,strides=1,activation=tf.nn.tanh)
conv5 = tf.layers.conv2d(conv4, filters=128, kernel_size=3,strides=1,activation=tf.nn.tanh)
#drop = tf.layers.dropout(conv5, .3, training=training)
fc=flatten(conv5)
print(conv5.shape)

fc1 = tf.layers.dense(fc, 80, activation=tf.nn.relu)
fc2 = tf.layers.dense(fc1, 80, activation=tf.nn.relu)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc2, n_outputs)
    Y_proba = tf.nn.softmax(logits)

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(50000 // batch_size):
            X_batch, y_batch = X_train[(batch_size*iteration):(batch_size*(iteration+1)),:], y_train[(batch_size*iteration):(batch_size*(iteration+1))]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val,y: y_val})
        print("Epoch",epoch,":", acc_val)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)

(?, 1, 1, 128)
Epoch 0 : 0.8542
Epoch 1 : 0.8789
Epoch 2 : 0.8886
Epoch 3 : 0.8994
Epoch 4 : 0.8933
Epoch 5 : 0.8984
Epoch 6 : 0.8956
Epoch 7 : 0.9027
Epoch 8 : 0.9006
Epoch 9 : 0.9038
Final accuracy on test set: 0.9066
